In [3]:
import os
import requests
from flask import Flask, render_template, request, redirect, url_for, send_from_directory
import SimpleITK as sitk
import numpy as np
import torch.cuda
import torch
from unet_3d import Unet
from utils import maxProb, FilterUnconnectedRegions, writeMhd, multilabel

In [4]:
app = Flask(__name__)

dataPath = '../../Data/LumbarSpine3D/InputImages/'
outputPath = '../../Data/LumbarSpine3D/InputImages/'
modelLocation = '../../Data/LumbarSpine3D/PretrainedModel/'
# Image format extension:
extensionImages = 'mhd'


if not os.path.exists(dataPath):
    os.makedirs(dataPath)

modelName = os.listdir(modelLocation)[0]
modelFilename = modelLocation + modelName


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
if device.type == 'cuda':
    t = torch.cuda.get_device_properties(0).total_memory
    r = torch.cuda.memory_reserved(0)
    a = torch.cuda.memory_allocated(0)
    f = r-a  # free inside reserved
    print('Total memory: {0}. Reserved memory: {1}. Allocated memory:{2}. Free memory:{3}.'.format(t,r,a,f))

cpu


In [6]:


######################### MODEL INIT ######################
multilabelNum = 8
torch.cuda.empty_cache()
model = Unet(1, multilabelNum)
model.load_state_dict(torch.load(modelFilename, map_location=device))
model = model.to(device)


In [14]:
radiusErodeDilate = sitk.VectorUInt32([1, 1, 1])  # Adjust the values as needed.


In [16]:

###################### READ DATA AND PRE PROCESS IT FOR TRAINING DATA SETS #####################################################
# Look for the folders or shortcuts:
files = os.listdir(dataPath)
files = sorted(files)
imageNames = []
imageFilenames = []
i = 0
for filename in files:
    name, extension = os.path.splitext(filename)
    if extension.endswith('raw'):
        continue
    filenameImage = dataPath + filename
    sitkImage = sitk.ReadImage('../../CNNs/unet/uploads_folder/C00001.mhd')
    image = sitk.GetArrayFromImage(sitkImage).astype(np.float32)
    image = np.expand_dims(image, axis=0)
    
    with torch.no_grad():
        input = torch.from_numpy(image).to(device)
        output = model(input.unsqueeze(0))
        output = torch.sigmoid(output.cpu().to(torch.float32))
        outputs = maxProb(output, multilabelNum)
        output = ((output > 0.5) * 1)
        output = multilabel(output.detach().numpy())
    #output = FilterUnconnectedRegions(output.squeeze(0), multilabelNum, sitkImage)# Herramienta de filtrado de imagenes
    #sitk.WriteImage(output, outputPath + name + '_segmentation' + extension)
    writeMhd(output.squeeze(0).astype(np.uint8), outputPath + name + '_segmentation' + extension, sitkImage) # sin herramienta de filtrado

RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\1\sitk-build\ITK\Modules\IO\Meta\src\itkMetaImageIO.cxx:484:
ITK ERROR: MetaImageIO(0000025373D5A3A0): File cannot be read: ../../CNNs/unet/uploads_folder/C00001.mhd for reading.
Reason: No such file or directory

In [40]:
import numpy as np
from PIL import Image

# Definir las dimensiones reales de tus datos
real_width = 640  # Ajusta según tus datos
real_height = 480  # Ajusta según tus datos

# Leer los datos raw en un arreglo NumPy
raw_file_path = r'C:\Users\DELL\MuscleSegmentation\Data\LumbarSpine3D\InputImages\C00001.raw'
with open(raw_file_path, 'rb') as raw_file:
    data = np.fromfile(raw_file, dtype=np.uint8)  # Ajusta el tipo de datos según sea necesario

# Verificar si es necesario recortar los datos
if real_width * real_height > len(data):
    raise ValueError("Las dimensiones reales son más grandes que la cantidad de datos disponibles.")

# Recortar los datos si es necesario
data = data[:real_width * real_height]

# Crear una imagen PIL y guardarla en otro formato
image_data = data.reshape((real_height, real_width))
image = Image.fromarray(image_data)
output_image_path = r'C:\Users\DELL\MuscleSegmentation\Data\LumbarSpine3D\InputImages\imagen.png'  # Especifica la ruta y nombre de archivo de salida
image.save(output_image_path)

print(f"Imagen guardada en '{output_image_path}'")


Imagen guardada en 'C:\Users\DELL\MuscleSegmentation\Data\LumbarSpine3D\InputImages\imagen.png'
